In [1]:
import sys, time, wmi, psutil
SYSTEM_INFO = wmi.WMI().Win32_OperatingSystem()[0]
"system: {0}, {1}, {2}".format(SYSTEM_INFO.Caption, SYSTEM_INFO.BuildNumber, SYSTEM_INFO.OSArchitecture) 
"memory: {}G".format(round(psutil.virtual_memory().total / 1024**3, 2))
"cpu: {}".format(psutil.cpu_count())
"python: {}".format(sys.version)
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))

'system: Microsoft Windows 10 教育版, 18363, 64 位'

'memory: 15.86G'

'cpu: 4'

'python: 3.7.1 (default, Oct 28 2018, 08:39:03) [MSC v.1912 64 bit (AMD64)]'

'2020-10-04 21:32:54'

- **@author**: run_walker
- **@references**:
    1. [functools.lru_cache装饰器](https://www.cnblogs.com/liao-lin/p/10855288.html)
    2. [functools.lru_cache装饰器详解](https://blog.csdn.net/wzqnls/article/details/78506022)

`functools.lru_cache`是非常实用的装饰器，它能把装饰的函数的结果保存起来，避免传入相同的参数时重复计算。***LRU***是***Least Recently Used***的缩写,表明缓存不会无限制增长，一段时间不用的缓存条目会被扔掉。

<div class="alert alert-block alert-warning">
    <i class="fa fa-sticky-note" aria-hidden="true"><b> Note:</b></i>
    <ul>
        <li>装饰的函数的参数必须可以被哈希。</li>
        <li>如果给参数`maxsize`传入None，则缓存没有空间限制，可以无限增长。</li>
    </ul>
</div>

In [3]:
import functools

In [4]:
help(functools.lru_cache)

Help on function lru_cache in module functools:

lru_cache(maxsize=128, typed=False)
    Least-recently-used cache decorator.
    
    If *maxsize* is set to None, the LRU features are disabled and the cache
    can grow without bound.
    
    If *typed* is True, arguments of different types will be cached separately.
    For example, f(3.0) and f(3) will be treated as distinct calls with
    distinct results.
    
    Arguments to the cached function must be hashable.
    
    View the cache statistics named tuple (hits, misses, maxsize, currsize)
    with f.cache_info().  Clear the cache and statistics with f.cache_clear().
    Access the underlying function with f.__wrapped__.
    
    See:  http://en.wikipedia.org/wiki/Cache_algorithms#Least_Recently_Used



In [6]:
import time
import functools


def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        cost_t = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst += [repr(arg) for arg in args]
        if kwargs:
            arg_lst += ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r ' % (cost_t, name, arg_str, result))
        return result
    return clocked

In [7]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

In [8]:
fibonacci(6)

[0.00000000s] fibonacci(0) -> 0 
[0.00000000s] fibonacci(1) -> 1 
[0.00400186s] fibonacci(2) -> 1 
[0.00000000s] fibonacci(1) -> 1 
[0.00000000s] fibonacci(0) -> 0 
[0.00000000s] fibonacci(1) -> 1 
[0.00299692s] fibonacci(2) -> 1 
[0.00599909s] fibonacci(3) -> 2 
[0.01401424s] fibonacci(4) -> 3 
[0.00000000s] fibonacci(1) -> 1 
[0.00000000s] fibonacci(0) -> 0 
[0.00000000s] fibonacci(1) -> 1 
[0.00299907s] fibonacci(2) -> 1 
[0.00499964s] fibonacci(3) -> 2 
[0.00000000s] fibonacci(0) -> 0 
[0.00000000s] fibonacci(1) -> 1 
[0.00200033s] fibonacci(2) -> 1 
[0.00000000s] fibonacci(1) -> 1 
[0.00000000s] fibonacci(0) -> 0 
[0.00000000s] fibonacci(1) -> 1 
[0.00199866s] fibonacci(2) -> 1 
[0.00400043s] fibonacci(3) -> 2 
[0.00900173s] fibonacci(4) -> 3 
[0.01599836s] fibonacci(5) -> 5 
[0.03199887s] fibonacci(6) -> 8 


8

可以看出使用递归会进行很多重复的计算，数据量增多时调用和计算更多。

In [9]:
@functools.lru_cache()
@clock
def fibonacci_lru(n):
    if n < 2:
        return n
    return fibonacci_lru(n - 2) + fibonacci_lru(n - 1)

In [10]:
fibonacci_lru(6)

[0.00000000s] fibonacci_lru(0) -> 0 
[0.00000000s] fibonacci_lru(1) -> 1 
[0.00400043s] fibonacci_lru(2) -> 1 
[0.00000000s] fibonacci_lru(3) -> 2 
[0.00800037s] fibonacci_lru(4) -> 3 
[0.00000000s] fibonacci_lru(5) -> 5 
[0.01000142s] fibonacci_lru(6) -> 8 


8

In [11]:
fibonacci_lru(6)  # 直接从缓存中取出结果

8

In [12]:
fibonacci_lru(8)

[0.00000000s] fibonacci_lru(7) -> 13 
[0.00299954s] fibonacci_lru(8) -> 21 


21

可以看到对出现过的参数，直接从缓存读取结果，不再重复计算。

In [13]:
fibonacci_lru.cache_info()

CacheInfo(hits=8, misses=9, maxsize=128, currsize=9)

In [17]:
fibonacci_lru.cache_clear()

In [18]:
fibonacci_lru.cache_info()

CacheInfo(hits=0, misses=0, maxsize=128, currsize=0)

In [19]:
fibonacci_lru(6)

[0.00000000s] fibonacci_lru(0) -> 0 
[0.00000000s] fibonacci_lru(1) -> 1 
[0.00197005s] fibonacci_lru(2) -> 1 
[0.00000000s] fibonacci_lru(3) -> 2 
[0.00496697s] fibonacci_lru(4) -> 3 
[0.00000000s] fibonacci_lru(5) -> 5 
[0.00796771s] fibonacci_lru(6) -> 8 


8

调用`func.cache_clear()`清空缓存后，再次调用会重新进行计算。

**总结**：`lru_cache`不能查看缓存的具体内容，在实际应用场景中这一点可能十分关键，所以更适合于一些很小的函数的递归实现，在leetcode刷题中经常会用到该功能；而对于一个需要查看当前已经解决过的所有情况的任务而言，需要自己手动建立一个哈希表，每次调用函数前判断其中是否已经存在了结果，如果没有的话，计算完毕后再将结果和输入参数的map更新进去。